In [1]:
import pandas as pd
import os
import re

In [2]:
file_path = "c:/Users/User/Desktop/Project/data_project/car_scrape_pipeline/dags/etl/data/raw/new_list.csv"
columns_keyword = ["year_produce", "brand", "model","car_type", "gear_type", "engine_cap","mileage", "price"]

In [3]:
try:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File {file_path} does not exist.")
    
    if file_path.endswith('.csv'):
        data = pd.read_csv(file_path, encoding='ISO-8859-1')
    elif file_path.endswith('.json'):
        data = pd.read_json(file_path)
    else:
        raise ValueError("Unsupported file format. Please provide a CSV or JSON file.")

    df = data

except FileNotFoundError as e:
    print(e)
except Exception as e:
    print(f"An error occurred: {e}")

In [4]:
df.rename(columns={'flex2': 'year_produce'}, inplace=True)

In [5]:
for col in df.columns:
    if 'Title1' in col:
        for index, row in df.iterrows():
            split_string = str(row["Title1"]).split(' ')
            df.at[index, 'brand'] = split_string[0]
            df.at[index, 'model'] = split_string[1].title()         

In [6]:
df.rename(columns={'flex': 'car_type'}, inplace=True)
df["gear_type"] = "Automatic"

In [7]:
for col in df.columns:
    if 'flex3' in col.lower():

        for index, row in df.iterrows():
            numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', row["flex3"])
            if numbers: 
                value = float(numbers[0])
                df.at[index, 'engine_cap'] = [round(value / 1000, 1) if value > 100 else round(value)]


In [8]:
for col in df.columns:
    if 'flex1' in col:

        for index, row in df.iterrows():
            numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', row["flex1"])
            if numbers:  
                numbers = [float(n.replace(",", "")) for n in numbers]

                if len(numbers) > 1:
                    avg_number = sum(numbers) / len(numbers)
                else:
                    avg_number = numbers[0]

                df.at[index, 'mileage'] = avg_number
            else:
                df.at[index, 'mileage'] = None

In [9]:
df['textsm'].fillna('0', inplace=True)

for col in df.columns:
    if 'textsm' in col.lower():    
        
        for index, row in df.iterrows():
            numbers = re.findall(r'\d+(?:,\d+)*(?:\.\d+)?', row['textsm'])
            if numbers:
                df.at[index, 'old_price'] = float(numbers[0].replace(',', ''))
            else:  
                df.at[index, 'old_price'] = 0

In [10]:
df = df[["year_produce", "brand", "model","car_type", "gear_type", "engine_cap","mileage", "old_price"]]
    
base_name = os.path.basename(file_path)
name, ext = os.path.splitext(base_name)
new_file_name = f"{name}_clean{ext}"
df.to_csv(f'{new_file_name}')